<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5173a9bd22697dc9d7edff79fffcd7a9b640ad6083ad8dcde70d516054d7fac4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-13 16:27:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -17.59 K (-0.12%)
Current PnL: -28.46 L (-18.15%)
CY Booked + Current PnL: -14.17 L (-9.03%)
-------------------
Total profit:  1.77 L
Total loss:  -30.22 L
-------------------
Total Booked + Current PnL: 12.81 L (9.87%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.16%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.91 L (66.78%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.48,16.26,4.42,21.40,13639.0,43150.0,308576.0,7.85,66.0,X-LC,6.14,14.0,3.16,2.19,23.61,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.79,5.47,13.04,19.23,20694.0,8232.0,158693.0,24.27,68.0,L-LC,8.30,182.0,0.40,1.13,27.91,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,2.14,-13.12,15.12,0.02,22932.0,-22895.0,151664.0,131.03,57.0,M-SC,8.52,240.0,-1.00,1.08,31.92,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.50,-22.71,29.51,0.09,22984.0,-22891.0,77886.0,80.79,32.0,M-SC,3.49,253.0,-1.00,0.55,2.33,OX40N,NTT,DURABLES
49,MASFIN,398.61,-0.46,-4.38,27.64,22.05,25896.0,-4290.0,93690.0,-17.34,46.0,H-SC,5.70,168.0,-0.17,0.67,36.19,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,4.20,-31.39,182.23,93.63,126229.0,-31694.0,69269.0,-26.56,50.0,H-SC,16.74,143.0,-0.25,0.49,4.72,SR,ATH,CHEMICALS
26,GILLETTE,11206.78,3.22,-4.36,38.46,32.42,93389.0,-11076.0,242820.0,-21.44,47.0,X-SC,12.30,87.0,-0.12,1.73,10.08,X40,ATH,FMCG
66,SIS,528.00,3.00,-23.11,57.71,21.26,49269.0,-25658.0,85374.0,2015.54,52.0,H-SC,4.71,163.0,-0.52,0.61,15.35,OX40N,NTT,MISC
13,BSOFT,831.70,2.84,-22.93,95.65,50.79,103685.0,-32253.0,108400.0,-1.04,50.0,H-SC,10.33,171.0,-0.31,0.77,26.21,XR,ATH,IT
63,SFL,1287.00,2.28,-43.95,125.93,26.63,185079.0,-115248.0,146970.0,11.25,40.0,M-SC,12.78,241.0,-0.62,1.05,2.28,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-5.10,-22.08,68.46,31.26,123097.0,-50960.0,179808.0,-63.58,26.0,X-MC,20.15,56.0,-0.41,1.28,0.00,X40N,ATH,IT
0,5PAISA,593.00,-3.37,-32.10,66.27,12.90,98322.0,-70130.0,148366.0,118.75,54.0,H-SC,9.74,173.0,-0.71,1.06,23.37,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,-2.63,-35.42,116.20,39.61,76913.0,-36310.0,66190.0,-56.41,46.0,H-SC,30.26,136.0,-0.47,0.47,7.14,AR,ATH,MISC
69,SURYODAY,216.00,-2.40,-20.45,54.66,23.03,77865.0,-36618.0,142453.0,55.70,47.0,H-SC,4.02,165.0,-0.47,1.01,41.34,XR,NTT,BANKS
18,COFFEEDAY,80.00,-2.32,-43.74,140.46,35.27,89723.0,-49671.0,63878.0,-57.75,40.0,L-SC,4.72,268.0,-0.55,0.45,56.34,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.97,-0.68,21.56,20.74,43507.0,-1375.0,201795.0,31.15,54.0,M-MC,5.48,189.0,-0.03,1.44,9.87,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.68,-3.51,115.47,107.91,164505.0,-5178.0,142466.0,-21.36,48.0,M-SC,10.17,220.0,-0.03,1.01,6.23,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.14,-13.12,15.12,0.02,22932.0,-22895.0,151664.0,131.03,57.0,M-SC,8.52,240.0,-1.00,1.08,31.92,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.75,-16.51,58.14,32.03,63993.0,-21768.0,110067.0,-9.66,67.0,H-SC,6.69,157.0,-0.34,0.78,23.28,OX40N,NTT,IT
47,KANSAINER,340.00,0.70,-22.48,46.39,13.47,96971.0,-60633.0,209034.0,-68.12,47.0,H-SC,4.36,159.0,-0.63,1.49,6.29,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.50,-22.71,29.51,0.09,22984.0,-22891.0,77886.0,80.79,32.0,M-SC,3.49,253.0,-1.00,0.55,2.33,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.79,5.47,13.04,19.23,20694.0,8232.0,158693.0,24.27,68.0,L-LC,8.30,182.0,0.40,1.13,27.91,XY25,ATH,MINING
84,WIPRO,420.00,0.42,8.36,58.97,72.27,108280.0,14171.0,183619.0,-6.99,53.0,M-LC,6.29,101.0,0.13,1.31,15.05,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.42,8.36,58.97,72.27,108280.0,14171.0,183619.0,-6.99,53.0,M-LC,6.29,101.0,0.13,1.31,15.05,XR,NTT,IT
25,FINCABLES,1641.55,0.68,-3.51,115.47,107.91,164505.0,-5178.0,142466.0,-21.36,48.0,M-SC,10.17,220.0,-0.03,1.01,6.23,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.13,-7.33,123.07,106.72,140669.0,-9036.0,114300.0,-54.06,37.0,H-SC,7.81,138.0,-0.06,0.81,14.34,AR,ATH,MISC
37,IEX,219.00,-0.50,-4.10,54.98,48.63,105429.0,-8196.0,191758.0,-36.24,51.0,H-SC,13.19,137.0,-0.08,1.36,7.04,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.63,-5.02,42.80,35.63,85196.0,-10519.0,199055.0,-18.51,40.0,H-MC,3.41,119.0,-0.12,1.42,10.82,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.09,-17.97,35.05,10.78,69097.0,-43180.0,197138.0,-50.96,13.0,X-LC,7.91,1.0,-0.62,1.40,0.00,X40,NTT,FMCG
7,AWL,485.00,-0.02,-28.77,120.68,57.19,280348.0,-93820.0,232307.0,-66.31,21.0,X-MC,10.00,58.0,-0.33,1.65,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-5.10,-22.08,68.46,31.26,123097.0,-50960.0,179808.0,-63.58,26.0,X-MC,20.15,56.0,-0.41,1.28,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.21,-41.43,130.86,35.20,98611.0,-53314.0,75356.0,-1.23,26.0,X-SC,17.51,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
78,UNITDSPR,1644.00,-0.62,-3.08,24.60,20.77,55827.0,-7207.0,226937.0,-11.53,29.0,X-MC,1.59,70.0,-0.13,1.61,3.63,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.62,-3.08,24.60,20.77,55827.0,-7207.0,226937.0,-11.53,29.0,X-MC,1.59,70.0,-0.13,1.61,3.63,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-1.27,-8.46,35.98,24.48,88063.0,-22609.0,244756.0,-10.02,36.0,X-LC,1.63,37.0,-0.26,1.74,41.87,XY24,BTT,FINANCE
20,DABUR,735.00,-0.37,2.07,40.79,43.70,103278.0,5126.0,253194.0,-6.31,65.0,X-MC,2.16,73.0,0.05,1.80,17.43,XY24,BTT,FMCG
65,SIEMENS,4671.50,-1.11,-20.80,58.48,25.51,86191.0,-38710.0,147385.0,-9.63,34.0,H-LC,2.18,50.0,-0.45,1.05,8.61,AR,ATH,ELECTRICAL
73,TCS,4389.97,0.88,-10.14,34.33,20.71,116678.0,-38365.0,339872.0,-22.65,55.0,X-LC,2.29,3.0,-0.33,2.42,13.56,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.02,-28.77,120.68,57.19,280348.0,-93820.0,232307.0,-66.31,21.0,X-MC,10.00,58.0,-0.33,1.65,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.21,-41.43,130.86,35.20,98611.0,-53314.0,75356.0,-1.23,26.0,X-SC,17.51,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-5.10,-22.08,68.46,31.26,123097.0,-50960.0,179808.0,-63.58,26.0,X-MC,20.15,56.0,-0.41,1.28,0.0,X40N,ATH,IT
43,ITC,452.00,-1.09,-17.97,35.05,10.78,69097.0,-43180.0,197138.0,-50.96,13.0,X-LC,7.91,1.0,-0.62,1.40,0.0,X40,NTT,FMCG
53,PGHH,17907.65,-0.10,-7.21,44.61,34.19,88385.0,-15392.0,198128.0,-34.79,31.0,X-MC,4.20,60.0,-0.17,1.41,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.48,-28.30,99.21,42.84,46244.0,-18394.0,46612.0,-53.17,49.0,X-SC,25.95,83.0,-0.40,0.33,5.46,XY24,NTT,MISC
58,RELAXO,1176.00,0.47,-48.76,197.23,52.31,146704.0,-70778.0,74382.0,-45.41,38.0,X-SC,7.82,92.0,-0.48,0.53,0.47,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.21,-41.43,130.86,35.20,98611.0,-53314.0,75356.0,-1.23,26.0,X-SC,17.51,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.14,-2.45,27.19,24.08,33672.0,-3110.0,123840.0,-10.30,45.0,X-SC,6.02,91.0,-0.09,0.88,16.69,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.32,-18.29,75.41,43.32,100356.0,-29792.0,133080.0,-29.79,41.0,X-SC,8.08,90.0,-0.30,0.95,2.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.09,-17.97,35.05,10.78,69097.0,-43180.0,197138.0,-50.96,13.0,X-LC,7.91,1.0,-0.62,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.23,-28.32,71.55,22.98,115864.0,-63964.0,161934.0,-25.52,38.0,X-LC,3.81,2.0,-0.55,1.15,1.85,XY24,NTT,AUTO
73,TCS,4389.97,0.88,-10.14,34.33,20.71,116678.0,-38365.0,339872.0,-22.65,55.0,X-LC,2.29,3.0,-0.33,2.42,13.56,X40,ATH,IT
81,VBL,671.64,1.50,1.24,34.01,35.67,108752.0,3924.0,319766.0,-10.77,60.0,X-LC,2.41,5.0,0.04,2.27,15.27,X40N,ATH,FMCG
41,INFY,1972.00,0.19,8.85,23.33,34.24,79459.0,27699.0,340587.0,-15.29,47.0,X-LC,7.58,6.0,0.35,2.42,17.87,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.56,-32.78,97.68,32.89,50697.0,-25306.0,51901.0,-741.04,65.0,L-MC,8.19,259.0,-0.50,0.37,42.93,XR,NTT,BANKS
13,BSOFT,831.70,2.84,-22.93,95.65,50.79,103685.0,-32253.0,108400.0,-1.04,50.0,H-SC,10.33,171.0,-0.31,0.77,26.21,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.14,-13.12,15.12,0.02,22932.0,-22895.0,151664.0,131.03,57.0,M-SC,8.52,240.0,-1.00,1.08,31.92,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.79,5.47,13.04,19.23,20694.0,8232.0,158693.0,24.27,68.0,L-LC,8.30,182.0,0.40,1.13,27.91,XY25,ATH,MINING
84,WIPRO,420.00,0.42,8.36,58.97,72.27,108280.0,14171.0,183619.0,-6.99,53.0,M-LC,6.29,101.0,0.13,1.31,15.05,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,1.75,-16.51,58.14,32.03,63993.0,-21768.0,110067.0,-9.66,67.0,H-SC,6.69,157.0,-0.34,0.78,23.28,OX40N,NTT,IT
0,5PAISA,593.00,-3.37,-32.10,66.27,12.90,98322.0,-70130.0,148366.0,118.75,54.0,H-SC,9.74,173.0,-0.71,1.06,23.37,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.79,5.47,13.04,19.23,20694.0,8232.0,158693.0,24.27,68.0,L-LC,8.30,182.0,0.40,1.13,27.91,XY25,ATH,MINING
57,RECLTD,446.00,-0.97,-0.68,21.56,20.74,43507.0,-1375.0,201795.0,31.15,54.0,M-MC,5.48,189.0,-0.03,1.44,9.87,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,0.56,-32.78,97.68,32.89,50697.0,-25306.0,51901.0,-741.04,65.0,L-MC,8.19,259.0,-0.50,0.37,42.93,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.44
1,25,44.62
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.60
MC    30.69
LC    26.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.28
X40      24.06
X40N     12.76
XY25     10.85
XR        8.94
AR        8.50
OX40N     7.65
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.30
X-LC    23.22
H-SC    22.79
M-SC    11.21
X-SC     7.84
H-MC     4.58
M-MC     1.44
M-LC     1.31
L-LC     1.13
H-LC     1.05
L-SC     0.76
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.28,-8.31,43.85
IT,12.88,-18.08,77.63
FINANCE,11.04,-14.99,63.26
MISC,7.05,-31.02,84.62
PAINTS,5.88,-11.84,28.45
ELECTRICAL,5.84,-14.53,56.48
INSURANCE,4.74,-1.78,36.61
PHARMA,3.97,-5.49,38.70
AUTO,2.77,-34.63,81.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3272708.0,21
AR,1377980.0,10
XR,1235257.0,12
X40,1140129.0,15
X40N,947942.0,9
OX40N,714961.0,10
XY25,416767.0,7
SR,285562.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3589311.0,24
X-MC,1617545.0,16
M-SC,1480893.0,15
X-LC,957220.0,13
X-SC,817701.0,8
H-MC,435908.0,3
L-SC,183359.0,2
M-LC,108280.0,1
H-LC,86191.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1262803.0      6
           AR         931820.0      5
M-SC       XY24       849837.0      6
H-SC       XR         796328.0      7
X-MC       X40        528725.0      7
           XY24       448120.0      3
           X40N       419405.0      4
X-LC       X40        419404.0      6
X-SC       X40N       325919.0      3
H-SC       OX40N      312798.0      4
M-SC       OX40N      308527.0      5
X-SC       XY24       299782.0      3
H-SC       SR         285562.0      2
H-MC       AR         227669.0      2
X-MC       XY25       221295.0      2
H-MC       XY24       208239.0      1
X-LC       XY24       203927.0      2
           X40N       202618.0      2
X-SC       X40        192000.0      2
M-SC       XR         190229.0      2
           AR         132300.0      2
X-LC       XY25       131271.0      3
M-LC       XR         108280.0      1
L-SC       OX40N       93636.0      1
           XR          89723.0      1
H-LC       AR          86191.0      1
L-MC       XR          50697.0      1
M-MC       XY25        43507.0      1
L-LC       XY25        20694.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
